In [1]:
import os

In [2]:
%pwd

'c:\\Users\\SAROURA\\OneDrive\\Documents\\ING4\\Semestre 2\\Machine Learning Avancé\\Projet MLOPS\\End-to-end-MLOps\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\SAROURA\\OneDrive\\Documents\\ING4\\Semestre 2\\Machine Learning Avancé\\Projet MLOPS\\End-to-end-MLOps'

In [ ]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Sarra006/End-to-end-MLOps.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Sarra006"
os.environ["MLFLOW_TRACKING_PASSWORD"]="81e47a205751d02495b1b9e080f75c391c08d1b8"

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    models_path: Path
    metrics_file_name: Path
    target_column: str
    mlflow_uri: str


In [7]:
from src.MLOpsProject.constants import *
from src.MLOpsProject.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            models_path = config.models_path,
            metrics_file_name = config.metrics_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/Sarra006/End-to-end-MLOps.mlflow",
        )
        return model_evaluation_config


In [9]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [10]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        accuracy = accuracy_score(actual, pred)
        precision = precision_score(actual, pred)
        recall = recall_score(actual, pred)
        f1 = f1_score(actual, pred)
        return accuracy, precision, recall, f1

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # Parcourir tous les modèles enregistrés
        model_files = [f for f in os.listdir(self.config.models_path) if f.endswith(('.pkl', '.joblib'))]
        results = {}

        for model_file in model_files:
            model_path = os.path.join(self.config.models_path, model_file)
            model = joblib.load(model_path)

            with mlflow.start_run(run_name=model_file):
                predictions = model.predict(test_x)

                accuracy, precision, recall, f1 = self.eval_metrics(test_y, predictions)

                # Enregistrer les métriques en local
                results[model_file] = {
                    "accuracy": accuracy,
                    "precision": precision,
                    "recall": recall,
                    "f1_score": f1
                }

                mlflow.log_param("model_file", model_file)
                mlflow.log_metrics({
                    "accuracy": accuracy,
                    "precision": precision,
                    "recall": recall,
                    "f1_score": f1
                })

                # Log du modèle
                if tracking_url_type_store != "file":
                    mlflow.sklearn.log_model(model, "model", registered_model_name=model_file.replace(".pkl", "").replace(".joblib", ""))
                else:
                    mlflow.sklearn.log_model(model, "model")

        # Sauvegarde globale des scores
        save_json(path=Path(self.config.metrics_file_name), data=results)


In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-05-03 17:01:07,675: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-03 17:01:07,681: INFO: common: yaml file: schema.yaml loaded successfully]


[2025-05-03 17:01:07,684: INFO: common: created directory at: artifacts]
[2025-05-03 17:01:07,686: INFO: common: created directory at: artifacts/model_evaluation]


2025/05/03 17:01:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'adaboost_model' already exists. Creating a new version of this model...
2025/05/03 17:01:24 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: adaboost_model, version 4
Created version '4' of model 'adaboost_model'.


🏃 View run adaboost_model.pkl at: https://dagshub.com/Sarra006/End-to-end-MLOps.mlflow/#/experiments/0/runs/e8ac51364a424c9f885771127692a3e1
🧪 View experiment at: https://dagshub.com/Sarra006/End-to-end-MLOps.mlflow/#/experiments/0


2025/05/03 17:01:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'decision_tree_model' already exists. Creating a new version of this model...
2025/05/03 17:01:36 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: decision_tree_model, version 4
Created version '4' of model 'decision_tree_model'.


...

🏃 View run random_forest_model.pkl at: https://dagshub.com/Sarra006/End-to-end-MLOps.mlflow/#/experiments/0/runs/740ef8875a374e90bc180649db7ee539
🧪 View experiment at: https://dagshub.com/Sarra006/End-to-end-MLOps.mlflow/#/experiments/0


2025/05/03 17:03:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'support_vector_machine_model' already exists. Creating a new version of this model...
2025/05/03 17:03:05 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: support_vector_machine_model, version 4
Created version '4' of model 'support_vector_machine_model'.


🏃 View run support_vector_machine_model.pkl at: https://dagshub.com/Sarra006/End-to-end-MLOps.mlflow/#/experiments/0/runs/858917c46e344ff79371c9285394339f
🧪 View experiment at: https://dagshub.com/Sarra006/End-to-end-MLOps.mlflow/#/experiments/0


2025/05/03 17:03:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'xgboost_model' already exists. Creating a new version of this model...
2025/05/03 17:03:21 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: xgboost_model, version 4
Created version '4' of model 'xgboost_model'.


🏃 View run xgboost_model.pkl at: https://dagshub.com/Sarra006/End-to-end-MLOps.mlflow/#/experiments/0/runs/f8e432fcf2274854bc30e0ec836fc3ef
🧪 View experiment at: https://dagshub.com/Sarra006/End-to-end-MLOps.mlflow/#/experiments/0
[2025-05-03 17:03:22,014: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
